In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load a pre-trained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the classifier with a new one, that has the same number of outputs as the number of classes in your dataset
num_classes = 2 # (number of classes, including background)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

import os
import torch
from PIL import Image

class UAVDataset(torch.utils.data.Dataset):
    def __init__(self, image_folder, label_folder, transforms=None):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.image_filenames = os.listdir(image_folder)
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(os.path.join(self.image_folder, self.image_filenames[index]))
        with open(os.path.join(self.label_folder, self.image_filenames[index].split('.')[0] + '.txt')) as f:
            label = [int(x) for x in f.read().split()]

        if self.transforms is not None:
            image = self.transforms(image)

        return image, torch.tensor(label)

    def __len__(self):
        return len(self.image_filenames)
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = UAVDataset(image_folder='/content/drive/MyDrive/DL/image', label_folder='/content/drive/MyDrive/DL/label', transforms=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2)

# Set up the optimizer and learning rate scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 10
for epoch in range(num_epochs):
    # Train one epoch
    model.train()
    for images, targets in data_loader:
        # Move the data to the GPU
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)

        # Backward pass
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()

        # Update the parameters
        optimizer.step()

    # Evaluate the model on the validation set
    model.eval()
    with torch.no_grad():
        for images, targets in val_data_loader:
            # Move the data to the GPU
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Forward pass
            outputs = model(images)

            # Calculate the loss
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
                    # Save the validation loss for this epoch
        val_loss = losses.item()

# Step the learning rate scheduler
lr_scheduler.step()
torch.save(model.state_dict(), 'faster_rcnn.pth')


ValueError: ignored